In [22]:
import cvxpy as cp
import numpy as np
from timeit import default_timer as timer

In [23]:
cost = np.array([2, 10, 5, 8, 7, 4, 2, 2, 2, 6, 2, 1, 7, 10, 4, 7, 1, 5, 10, 4, 1, 1, 1, 3, 2, 2, 3, 7, 3, 5, 3, 3, 0])
time = np.array([1, 4, 3, 6, 6, 2, 1, 3, 2, 5, 3, 2, 2, 5, 6, 4, 5, 4, 3, 2, 4, 3, 2, 1, 2, 2, 2, 1, 3, 5, 6, 4, 3])
profit = np.array([0, 3, 1, 3, 3, 0, 0, 0, 0, 2, 1, 0, 2, 3, 2, 2, 1, 2, 2, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 1, 1])
area = np.array([2, 5, 8, 6, 4, 6, 6, 7, 5, 4, 5, 6, 6, 6, 4, 6, 6, 5, 5, 4, 7, 3, 5, 3, 4, 3, 4, 5, 4, 5, 6, 5, 6])
button_income_placements = np.array([4, 10, 16, 22, 28, 34, 40, 46, 52])

In [24]:
X = cp.Variable((33, 9), boolean=True)
Y = cp.Variable((33, 9), boolean=True)
z = cp.Variable(33, boolean=True)
psi = cp.Variable(9, integer=True)
xi = cp.Variable(8, integer=True)
lambd = cp.Variable(1, boolean=True)
omega = cp.Variable(33, boolean=True)

In [25]:
def get_column(matrix, column):
    return matrix[:,column-1]

def get_row(matrix, row):
    return matrix[row-1,:]

def index(matrix, rowOrColumn, column=None):
    if column is None:
        return matrix[rowOrColumn-1]

    return matrix[rowOrColumn-1][column-1]

def range_index(matrix, start, end):
    return matrix[start - 1:end]

def vector(number, amount):
    return np.repeat(number, amount)

def foreach(start, end):
    return range(start, end + 1)

In [26]:
constraints = []

# Area 
constraints.extend([
    area @ get_column(X, 9) <= 81,
    area @ get_column(X, 9) - 76 + 1 <= 1000 * lambd,
    76 * lambd <= area @ get_column(X, 9),
])

# Time
constraints.extend([
    time @ (get_column(X, 9) - z) + psi @ vector(1, 9) <= 52,
    z @ vector(1, 33) <= 1,
])
for i in foreach(1, 33):
    constraints.extend([
        index(z, i) <= index(X, i, 9) - index(X, i, 8),
    ])

# Button Income
constraints.extend([
    (get_column(X, 1) - get_column(Y, 1)) @ time + index(psi, 1) <= index(button_income_placements, 1)
])
for j in foreach(2,9):
    constraints.extend([
        index(xi, j-1) + (get_column(X, j) - get_column(X, j-1) - get_column(Y, j)) @ time + index(psi, j) <= index(button_income_placements, j)
    ])

constraints.extend([
    index(xi, 1) >= time @ get_column(X, 1) + index(psi, 1)    
])
for j in foreach(2,8):
    constraints.extend([
        index(xi, j) >= index(xi, j-1) + time @ (get_column(X, j) - get_column(X, j-1)) + index(psi, j)
    ])

for j in foreach(1,8):
    constraints.extend([
        index(xi, j) >= index(button_income_placements, j)
    ])

for j in foreach(1,9):
    constraints.extend([
        get_column(Y, j) @ vector(1, 33) <= 1,
    ])

for i in foreach(1, 33):
    constraints.extend([
        index(Y, i, 1) <= index(X, i, 1),
    ])
    
    for j in foreach(2,9):
        constraints.extend([
            index(Y, i, j) <= index(X, i, j) - index(X, i, j-1)
        ])

constraints.extend([
    5 >= cost @ get_column(X, 1), 
    5 + profit @ get_column(X, 1) + index(psi, 1) >= cost @ get_column(X, 2), 
    5 + profit @ get_column(X, 1) + profit @ get_column(X, 2) + index(psi, 1) + index(psi, 2) >= cost @ get_column(X, 3), 
    5 + profit @ get_column(X, 1) + profit @ get_column(X, 2) + profit @ get_column(X, 3) + index(psi, 1) + index(psi, 2) + index(psi, 3) >= cost @ get_column(X, 4), 
    5 + profit @ get_column(X, 1) + profit @ get_column(X, 2) + profit @ get_column(X, 3) + profit @ get_column(X, 4) + index(psi, 1) + index(psi, 2) + index(psi, 3) + index(psi, 4) >= cost @ get_column(X, 5), 
    5 + profit @ get_column(X, 1) + profit @ get_column(X, 2) + profit @ get_column(X, 3) + profit @ get_column(X, 4) + profit @ get_column(X, 5) + index(psi, 1) + index(psi, 2) + index(psi, 3) + index(psi, 4) + index(psi, 5) >= cost @ get_column(X, 6), 
    5 + profit @ get_column(X, 1) + profit @ get_column(X, 2) + profit @ get_column(X, 3) + profit @ get_column(X, 4) + profit @ get_column(X, 5) + profit @ get_column(X, 6) + index(psi, 1) + index(psi, 2) + index(psi, 3) + index(psi, 4) + index(psi, 5) + index(psi, 6) >= cost @ get_column(X, 7), 
    5 + profit @ get_column(X, 1) + profit @ get_column(X, 2) + profit @ get_column(X, 3) + profit @ get_column(X, 4) + profit @ get_column(X, 5) + profit @ get_column(X, 6) + profit @ get_column(X, 7) + index(psi, 1) + index(psi, 2) + index(psi, 3) + index(psi, 4) + index(psi, 5) + index(psi, 6) + index(psi, 7) >= cost @ get_column(X, 8), 
    5 + profit @ get_column(X, 1) + profit @ get_column(X, 2) + profit @ get_column(X, 3) + profit @ get_column(X, 4) + profit @ get_column(X, 5) + profit @ get_column(X, 6) + profit @ get_column(X, 7) + profit @ get_column(X, 8) + index(psi, 1) + index(psi, 2) + index(psi, 3) + index(psi, 4) + index(psi, 5) + index(psi, 6) + index(psi, 7) + index(psi, 8) >= cost @ get_column(X, 9), 
])

for i in foreach(1,33):
    for j in foreach(1,8):
        constraints.extend([
            index(X, i, j) <= index(X, i, j + 1)
        ])

for j in foreach(1, 9):
    constraints.extend([
        index(psi, j) >= index(button_income_placements, j) - time @ get_column(X, j)
    ])

for i in foreach(1,33):                                                    
    constraints.extend([
        index(omega, i) >= index(X, i, 9)
    ])

constraints.extend([
    omega @ vector(1, 33) <= 33 * lambd    
])

for i in foreach(1, 33):
    constraints.extend([
        index(omega, i) >= index(X, i, 9) + lambd - 1
    ])

"""
b = cp.Variable(8, integer=True)

constraints.extend([
    index(xi, 1) <= index(button_income_placements, 1) + 60 * index(b, 1),
    index(xi, 1) <= index(xi, j-1) + time @ (get_column(X, j)  get_column(X, j-1)) + index(psi, j) + 60 * (1 - index(b, 1)),
])
for j in foreach(2,8):
    constraints.extend([
        index(xi, j) <= index(button_income_placements, j) + 60 * index(b, j),
        index(xi, j) <= index(xi, j-1) + time @ (get_column(X, j)  get_column(X, j-1)) + index(psi, j) + 60 * (1 - index(b, j)),
    ])
"""

constraints.extend([
    0 <= psi,
    psi <= 500,
    0 <= xi,
    xi <= 500,
])

In [27]:
objective = cp.Maximize(
    5 + 7 - 81 * 2
    + profit @ X @ vector(1, 9)
    + 2 * 81 * lambd
    + 2 * area @ get_column(X, 9)
    - 2 * area @ omega
    - cost @ get_column(X, 9)
    + psi @ vector(1,9)
)

In [28]:
problem = cp.Problem(objective, constraints)
fast = True

# https://www.cvxpy.org/tutorial/advanced/index.html Solver args

kwargs = None,
if fast:
    kwargs = {
        'max_iters': 100, # maximum number of iterations (default: 100).
        'abstol': 1e-7, # absolute accuracy (default: 1e-7).
        'reltol': 1e-6, # relative accuracy (default: 1e-6).
        'feastol': 1e-7, # tolerance for feasibility conditions (default: 1e-7).
    }
else:
   kwargs = {
        'max_iters': 250,
        'abstol': 1e-10, 
        'reltol': 1e-10, 
        'feastol': 1e-10,
    } 

start = timer()
result = problem.solve(verbose=True, **kwargs)
end = timer()

print(f"Problem status: {problem.status}")
print(problem.solver_stats)
print(result)
print(f'Solver took {end - start}s')

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Mar 25 08:54:51 PM: Your problem has 678 variables, 722 constraints, and 0 parameters.
(CVXPY) Mar 25 08:54:51 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 25 08:54:51 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 25 08:54:51 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 25 08:54:51 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 25 08:54:52 PM: Compiling problem (target solver=GLPK_MI)

In [29]:
np.set_printoptions(
    #edgeitems=30, 
    linewidth=100000, 
    #formatter=dict(float=lambda x: "%.3g" % x)
)

print(f"Value = {result}")
print(f"   Start    = {5}");
print(f"   SpecTile = {7}");
print(f"   Profit   = {profit @ X.value @ vector(1,9)}")
print(f"   Std Area = {-2 * 81}");
print(f"   Area     = {2 * 81 * lambd.value.item() + 2 * area @ get_column(X.value, 9) - 2 * area @ omega.value}")
print(f"       Area1    = {2 * 81 * lambd.value.item()}")
print(f"       Area2    = {2 * area @ get_column(X.value, 9)}")
print(f"       Area3    = {- 2 * area @ omega.value}")
print(f"   Cost     = {- cost @ get_column(X.value, 9)}")
print(f"   Walk     = {psi.value @ vector(1,9)}")

print(f"patches             = {np.argwhere(get_column(X.value, 9)).squeeze()}")
area_cost = get_column(X.value, 9) * area;
print(f"patches area        = {area_cost[area_cost > 0]} (sum = {area_cost[area_cost > 0].sum()})")
time_cost = get_column(X.value, 9) * time;
print(f"patches time cost   = {time_cost[time_cost > 0]} (sum = {time_cost[time_cost > 0].sum()})")
button_cost = get_column(X.value, 9) * cost;
print(f"patches button cost = {button_cost[button_cost > 0]} (sum = {button_cost[button_cost > 0].sum()})")

print(f"patches [1]         = {np.argwhere(get_column(X.value, 1)).squeeze()}")
print(f"patches [2]         = {np.argwhere(get_column(X.value, 2)).squeeze()}")
print(f"patches [3]         = {np.argwhere(get_column(X.value, 3)).squeeze()}")
print(f"patches [4]         = {np.argwhere(get_column(X.value, 4)).squeeze()}")
print(f"patches [5]         = {np.argwhere(get_column(X.value, 5)).squeeze()}")
print(f"patches [6]         = {np.argwhere(get_column(X.value, 6)).squeeze()}")
print(f"patches [7]         = {np.argwhere(get_column(X.value, 7)).squeeze()}")
print(f"patches [8]         = {np.argwhere(get_column(X.value, 8)).squeeze()}")
print(f"patches [9]         = {np.argwhere(get_column(X.value, 9)).squeeze()}")

print(f"X[9] = {get_column(X.value, 9)}")
print(f"X^T =\n{X.value.T}")
print(f"Y^T =\n{Y.value.T}")
print(f"z = {z.value}")
print(f"ψ = {psi.value}")
print(f"ξ = {xi.value}")
print(f"λ = {lambd.value}")
print(f"ω = {omega.value}")
print("\ngiven constants")
print(f"b = {button_income_placements}")
print(f"t = {time}")
print(f"p = {profit}")
print(f"a = {area}")
print(f"c = {cost}")

Value = 84.0
   Start    = 5
   SpecTile = 7
   Profit   = 116.0
   Std Area = -162
   Area     = 162.0
       Area1    = 162.0
       Area2    = 154.0
       Area3    = -154.0
   Cost     = -51.0
   Walk     = 7.0
patches             = [ 1  3  4  6 10 11 12 16 17 20 22 26 30 32]
patches area        = [5. 6. 4. 6. 5. 6. 6. 6. 5. 7. 5. 4. 6. 6.] (sum = 77.0)
patches time cost   = [4. 6. 6. 1. 3. 2. 2. 5. 4. 4. 2. 2. 6. 3.] (sum = 50.0)
patches button cost = [10.  8.  7.  2.  2.  1.  7.  1.  5.  1.  1.  3.  3.] (sum = 51.0)
patches [1]         = [17 32]
patches [2]         = [17 30 32]
patches [3]         = [ 4 17 30 32]
patches [4]         = [ 3  4 17 30 32]
patches [5]         = [ 1  3  4 17 30 32]
patches [6]         = [ 1  3  4  6 10 12 17 30 32]
patches [7]         = [ 1  3  4  6 10 12 17 20 26 30 32]
patches [8]         = [ 1  3  4  6 10 11 12 17 20 22 26 30 32]
patches [9]         = [ 1  3  4  6 10 11 12 16 17 20 22 26 30 32]
X[9] = [0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0.

In [46]:
mul = profit * get_column(X.value, 9)
print(mul)
print(mul[mul == 1].shape)
print(mul[mul == 2].shape)
print(mul[mul == 3].shape)

[0. 3. 0. 3. 3. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 0. 1. 2. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 2. 0. 1.]
(5,)
(3,)
(3,)
